In [1]:
!pip install sentence-transformers underthesea

  Using cached sentence_transformers-3.4.1-py3-none-any.whl.metadata (10 kB)
  Using cached underthesea-6.8.4-py3-none-any.whl.metadata (15 kB)
  Using cached nltk-3.9.1-py3-none-any.whl.metadata (2.9 kB)
Using cached sentence_transformers-3.4.1-py3-none-any.whl (275 kB)
   ---------------------------------------- 0.0/20.9 MB ? eta -:--:--
    --------------------------------------- 0.3/20.9 MB ? eta -:--:--
   -- ------------------------------------- 1.0/20.9 MB 2.6 MB/s eta 0:00:08
   -- ------------------------------------- 1.3/20.9 MB 2.5 MB/s eta 0:00:08
   --- ------------------------------------ 1.6/20.9 MB 2.3 MB/s eta 0:00:09
   --- ------------------------------------ 1.8/20.9 MB 1.7 MB/s eta 0:00:11
   --- ------------------------------------ 1.8/20.9 MB 1.7 MB/s eta 0:00:11
   --- ------------------------------------ 1.8/20.9 MB 1.7 MB/s eta 0:00:11
   ---- ----------------------------------- 2.1/20.9 MB 1.2 MB/s eta 0:00:16
   ---- ----------------------------------- 2.1/2


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from tqdm import tqdm
import sys
from underthesea import word_tokenize
import pandas as pd
import pickle

c:\Users\DELL\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
%pwd

'e:\\Legal-document-dual-retrieval-approach\\tokenize'

In [7]:
%cd ..

e:\Legal-document-dual-retrieval-approach


C:\Users\DELL\AppData\Roaming\Python\Python311\site-packages\IPython\core\magics\osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [8]:
%pwd

'e:\\Legal-document-dual-retrieval-approach'

In [ ]:
corpus = pd.read_csv('data/corpus.csv')
train = pd.read_csv('data/train.csv', on_bad_lines='skip')

In [4]:
nan_qid = train[train['qid'].isna() == True].index
nan_qid

Index([26402], dtype='int64')

In [5]:
train.loc[nan_qid[0], 'qid'] = 0
train['qid'] = train['qid'].astype(int)
train.loc[nan_qid[0], 'qid']

0

In [6]:
train.head()

,question,context,cid,qid
0,Người học ngành quản lý khai thác công trình t...,"['Khả năng học tập, nâng cao trình độ\n- Khối ...",[62492],161615
1,Nội dung lồng ghép vấn đề bình đẳng giới trong...,['Nội dung lồng ghép vấn đề bình đẳng giới tro...,[151154],80037
2,Sản phẩm phần mềm có được hưởng ưu đãi về thời...,"['""Điều 20. Ưu đãi về thời gian miễn thuế, giả...",[75071],124074
3,Điều kiện để giáo viên trong cơ sở giáo dục mầ...,"['Điều kiện được hưởng\nCán bộ quản lý, giáo v...",[225897],146841
4,Nguyên tắc áp dụng phụ cấp ưu đãi nghề y tế th...,"['Nguyên tắc áp dụng\n1. Trường hợp công chức,...",[68365],6176


In [7]:
train['cid'][:10]

0          [62492]
1         [151154]
2          [75071]
3         [225897]
4          [68365]
5         [161160]
6          [73585]
7    [31682 31677]
8          [64594]
9          [92952]
Name: cid, dtype: object

In [8]:
relevant = train['cid'].apply(lambda x: len(x.strip('[]').split())).tolist()
relevant[:10]

[1, 1, 1, 1, 1, 1, 1, 2, 1, 1]

In [9]:
train['list_cid'] = train['cid'].apply(lambda x: x.strip('[]').split()).tolist()
train['list_cid'] = train['list_cid'].apply(lambda x: [int(i) for i in x])

In [10]:
train['list_cid'][:10]

0           [62492]
1          [151154]
2           [75071]
3          [225897]
4           [68365]
5          [161160]
6           [73585]
7    [31682, 31677]
8           [64594]
9           [92952]
Name: list_cid, dtype: object

In [11]:
import numpy as np
np.argmax(relevant)

26402

In [12]:
documents_dir = dict(zip(corpus['cid'], corpus['text']))
queries_dir = dict(zip(train['qid'], train['question']))
queries_doc = dict(zip(train['qid'], train['list_cid']))
queries_map = dict(zip(train['question'], train['qid']))

In [14]:
documents_id = list(documents_dir.keys())
documents_list = list(documents_dir.values())

In [15]:
queries_id = list(queries_dir.keys())
queries_list = list(queries_dir.values())

In [16]:
tokenized_documents = [word_tokenize(doc) for doc in tqdm(documents_list, file=sys.stdout)]

100%|██████████| 261597/261597 [2:15:38<00:00, 32.14it/s]


In [17]:
with open('tokenized_documents.pkl', 'wb') as f:
    pickle.dump(tokenized_documents, f)